In [44]:
import math
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from inverse_covariance import QuicGraphicalLassoEBIC, QuicGraphicalLasso
from sklearn.covariance import GraphicalLasso
from nilearn import plotting

import io
import dash
import dash_table
import dash_html_components as html
import base64

from jupyter_dash import JupyterDash 
import dash_cytoscape as cyto
from dash.dependencies import Input, Output, State
import dash_core_components as dcc

In [45]:
stylesheet = [{
        "selector": 'node',
        'style': {
            'position': 'absolute',
            'width': '30%',
            'height': '30%',
            "content": "data(label)",
            "font-size": "10px",
            "text-valign": "center",
            "text-halign": "center",
            "background-color": "#555",
            "text-outline-color": "#555",
            "text-outline-width": "2px",
            "color": "#FFFFFF",
            "overlay-padding": "6px",
        }
    },
    
    
    {
      "selector": "edge[weight<0]",
      "style": {
        "curve-style": "haystack",
        "haystack-radius": "0.5",
        "line-color": "#E85655",
        "width": "mapData(weight, -1, 0, 20, 1)",
        "opacity": "mapData(weight, -1, 0, 1, 0)",
  }
},
    
    {
      "selector": "edge[weight>0]",
      "style": {
        "curve-style": "haystack",
          "haystack-radius": "0.5",
        "line-color": "#6E8BC7",
        "width": "mapData(weight, 0, 1, 1, 20)",
        "opacity": "mapData(weight, 0, 1, 0, 1)",
  }
},

    
]

In [46]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)
app.title = 'Network Capstone Design'

app.layout = html.Div([
    dcc.Upload(id = 'upload-data', 
        children = html.Div(['Drag and Drop or ', html.A('Select Files')]), 
               
        style = {
        'width': '100%',
        'height': '60px',
        'lineHeight': '60px',
        'borderWidth': '1px',
        'borderStyle': 'dashed',
        'borderRadius': '5px',
        'textAlign': 'center',
        'margin': '10px'
    },
        multiple=True          
    ),
               
    cyto.Cytoscape(id = 'cytoscape',
        style = {
        'width': '100%',
        'height': '550px'
    }, 
                   
        layout = {
        'name': 'cose',
        'idealEdgeLength': 100,
        'nodeOverlap': 5,
        'refresh': 20,
        'fit': True,
        'padding': 10,
        'randomize': True,
        'componentSpacing': 200,
        'nodeRepulsion': 400000,
        'edgeElasticity': 100,
        'nestingFactor': 5,
        'gravity': 1,
        'numIter': 1000,
        'initialTemp': 200,
        'coolingFactor': 0.95,
        'minTemp': 1.0
    }, 
    
    stylesheet = stylesheet, 
    responsive = True),
        
    html.Div(id = "output-data-upload"),

])

In [47]:
@app.callback(
    [Output("output-data-upload", "children"), Output('cytoscape', 'elements')],
    [Input("upload-data", "contents"), Input("upload-data", "filename")])


def update_table(contents, filename):
    if contents:
        contents= contents[0]
        filename= filename[0]

        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)

        try:
            if "csv" in filename:
                df= pd.read_csv(io.StringIO(decoded.decode("utf-8")))
            elif "xls" or "xlsx" in filename:
                df= pd.read_excel(io.BytesIO(decoded))
            elif "txt" or "tsv" in filename:
                df= pd.read_csv(io.StringIO(decoded.decode("utf-8")), delimiter=r"\s+")
        except Exception as e:
            print(e)
            return html.Div(["There was an error processing this file."])
        
        df = df.dropna(axis=1, how='all')
        df = df.dropna(axis=0, how='any')
        columnName = list(df.columns.values)
        
        best_alpha = Compute_BEST_GGM_from_extended_BIC(df, 0.1)
        estimator = QuicGraphicalLasso(lam=best_alpha, auto_scale =True, verbose=1, 
                                       init_method='spearman', path=None,
                                       max_iter=10000, method='quic')
        estimator.fit(df.values)

        df = pd.DataFrame(np.triu(-cov2corr(estimator.precision_),1))
        result = df
        result.to_csv (r'./mydf.csv', index = True, header=columnName)
        
        df.columns = columnName
        df.index = columnName
        
        table = html.Div(
            [
                
                dash_table.DataTable(
                    columns=[{"name": str(i), "id": str(i)} for i in df.columns],
                    data=df.to_dict("records"),
                    style_data={
                        'whiteSpace': 'normal',
                        'height': 'auto',
                    },
                ),
                
            ],
        )
    
        
        df = pd.DataFrame.from_dict(df)
        df = df.stack().reset_index()

        df.columns = ['from', 'to', 'weight']
        nodes = set()  
        
        cy_edges = []
        cy_nodes = []

        
        for name in columnName:
            nodes.add(name)
            cy_nodes.append({"data": {"id":name, "label": name}})
            
        for index, row in df.iterrows():
            source, target, weight = row['from'], row['to'], row['weight']
    
            if(weight != 0):
                #nodes.add(source)
                #cy_nodes.append({"data": {"id": source, "label": source}})
                #nodes.add(target)
                #cy_nodes.append({"data": {"id": target, "label": target}})

                cy_edges.append({
                    'data': {
                        'source': source,
                        'target': target,
                        'weight': weight
                    }
                })
    

        elements = cy_edges+cy_nodes
        
        print(elements)
        
        return table, elements

    else:
        return {}, {}


app.run_server(mode='external')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


Dash app running on http://127.0.0.1:8050/


In [4]:
def skeptic(x):
    y = 2 * math.sin(x * math.pi/6.)
    return y

def npn(x):
    df = x.corr(method = "spearman")
    result = df.applymap(skeptic)

    return result

In [5]:
def lamSet(tr_):
    lamMax = max(np.max(tr_.values), -np.min(tr_.values))
    
    lamMin = 0.01 * lamMax
    lamMaxX = math.log(lamMax)
    lamMinX = math.log(lamMin)
    
    lam = np.logspace(np.log10(lamMin), np.log10(lamMax), 100)[::-1]
    return lam

In [6]:
from sklearn.utils.extmath import fast_logdet

def Compute_extended_BIC(S, K, n, p, gamma):
    #S->cov  K-> model(res$wi)
    
    L=n*0.5*(np.log(np.linalg.det(K))-np.trace(np.dot(K, S)))
    E=(np.sum(np.sum(K != 0, axis=0))-p)*0.5
    EBIC=-2*L+E*np.log(n)+4*E*gamma*np.log(p)

    return EBIC

In [9]:
def Compute_BEST_GGM_from_extended_BIC(X,gamma):
    
    tr = npn(X)
    tr = tr.values
    tr_ = pd.DataFrame(np.triu(tr, 1))
    cov=np.cov(X.T)
    
    n=X.shape[0]
    p=X.shape[1]
    
    lam = lamSet(tr_)
    num_alphas=lam.shape[0]
    EBICs=np.zeros(num_alphas)
     
    for i in range(num_alphas):
        alpha= round(lam[i], 9)
        model = QuicGraphicalLasso(lam=alpha, auto_scale =False,
                                    tol=0, init_method='spearman', path=None, 
                                    max_iter=100, method='quic').fit(X)
        
        EBIC=Compute_extended_BIC(tr, model.precision_, n, p, gamma)
        #EBIC = ebic(tr, model.precision_, p, n, 0.1)
        print("EBIC :"+str(EBIC)+" alpha :"+str(alpha)+" model: "+str(model.lam_))
        EBICs[i]=EBIC
         
    #plt.plot(lam, EBICs)
    #plt.xlabel("L1 regularization parameter (alpha)")
    #plt.ylabel("extended BIC (EBIC)")
    #plt.show()
    
    best_idx=np.argmin(EBICs)
    best_alpha=lam[best_idx]
    return float(best_alpha)

In [10]:
def cov2corr( A ):
    d = np.sqrt(A.diagonal())
    A = ((A.T/d).T)/d
    A[ np.diag_indices(A.shape[0]) ] = np.ones( A.shape[0] )
    return A